In [1]:
!pip install tensorflow-text==2.9.0
!pip install py_vncorenlp
!pip install tokenization
!pip install vncorenlp
!pip install fairseq
!pip install demoji
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 10.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 38.8 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Succ

In [2]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2022-12-01 01:18:18--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2022-12-01 01:18:20 (185 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-12-01 01:18:21--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [3]:
import regex
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

2022-12-01 01:18:27.436779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-01 01:18:27.436850: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
from vncorenlp import VnCoreNLP

# rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [5]:
# text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
# output = rdrsegmenter.tokenize(text)
# output = ' '.join([' '.join(x) for x in output])
# print(output)

# EDA

In [6]:
import pandas as pd

# df = pd.read_csv('Dataset/full_train.csv')
df = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/full_train.csv')

In [12]:
df

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
0,0,3839333,10106093.0,"xôi dẻo, đồ ăn đậm vị. hộp xôi được lót lá trô...",['https://images.foody.vn/res/g97/966781/s800/...,1.0
1,1,2824877,786914.0,gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,['https://images.foody.vn/res/g69/688413/s800/...,0.0
2,2,9816702,22467889.0,"thời tiết lạnh như này, cả nhà rủ nhau đến leg...",['https://images.foody.vn/res/g72/715078/s800/...,1.0
3,3,2684585,1889449.0,em có đọc review thấy mng bảo trà sữa nướng đề...,['https://images.foody.vn/res/g90/895545/s800/...,0.0
4,4,2737987,8839942.0,"đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",['https://images.foody.vn/res/g4/30186/s800/fo...,1.0
...,...,...,...,...,...,...
9068,9066,11236510,23044142.0,thực sự mà nói thấy mọi người đánh giá nhiều q...,['https://images.foody.vn/res/g94/930433/s800/...,1.0
9069,9067,1831645,10117823.0,👉👉👉lẩu thái tômyum\n👉👉đc: 19 lò đúc-hbt-hn\n👉t...,['https://images.foody.vn/res/g16/150633/s800/...,1.0
9070,9068,2155495,11636069.0,ngay từ lúc đầu tiên bước vào nhà hàng đã được...,['https://images.foody.vn/res/g74/736118/s800/...,1.0
9071,9069,7357032,12208284.0,đặt ăn thử mà thấy ngón cá...! 🤩 sẽ còn ủn hộ ...,['https://images.foody.vn/res/g104/1036659/s80...,1.0


In [8]:
# df.to_csv('data_foody_raw.csv', encoding = 'utf-8-sig')

In [13]:
df['Comment'] = df['Comment'].str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df = df[df['Comment'].notnull()]

In [15]:
df[df['Comment'].str.contains('ngon tuyệt vời')]

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
3403,3401,4206281,13333873.0,mì có 64 nghìn mà đầy ự luôn ạ!\nngon tuyệt vờ...,['https://images.foody.vn/res/g101/1003994/s80...,1.0
3478,3476,12698037,23605066.0,"nộm bò kho ngon tuyệt vời , sợi dài , giòn , n...",['https://images.foody.vn/res/g108/1073618/s80...,1.0
4392,4390,3339491,1354106.0,📍 về chất lượng: \nriêng về chất lượng thì sum...,['https://images.foody.vn/res/g1/6921/s800/foo...,1.0
4586,4584,2032833,9308096.0,"quán chuyêm buffet lẩu với các mức giá 199k, 2...",['https://images.foody.vn/res/g72/712575/s800/...,1.0
4871,4869,256085,180584.0,ra ăn lần đầu do được mời và được quảng cáo là...,['https://images.foody.vn/res/g5/42464/s800/fo...,1.0
5062,5060,5129738,19264980.0,"bánh thơm ngon tuyệt vời.độ ngọt vừa phải,màu ...",['https://images.foody.vn/res/g102/1015725/s80...,1.0
5285,5283,2130580,2021276.0,hôm nọ mình và người yêu lặn lội lên tận phan ...,['https://images.foody.vn/res/g74/739263/s800/...,1.0
5901,5899,2639792,15203675.0,"món ngon tuyệt vời, giá cà hợp lý",['https://images.foody.vn/res/g5/46542/s800/fo...,1.0
6448,6446,3948192,1365299.0,mình ăn ở đây cũng được vài tháng. lúc đầu quá...,['https://images.foody.vn/res/g99/989770/s800/...,1.0
7464,7462,396775,818169.0,chỗ này ăn mấy năm nay rồi mà không hiểu sao c...,['https://images.foody.vn/res/g18/172897/s800/...,1.0


In [12]:
# ngon tuyet vời 100% tích cực

In [17]:
# df[(df['Comment'].str.contains('sẽ quay lại')) & (df['Rating'] < 6.0)]['Comment'].to_list()[4]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = df["Comment"].to_list()
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X = vectorizer.fit_transform(corpus)

In [19]:
vectorizer.get_feature_names_out()[207000:]

array(['ổn cái', 'ổn còn', 'ổn có', 'ổn cút', 'ổn cũng', 'ổn cơ',
       'ổn cơm', 'ổn cả', 'ổn cảm', 'ổn cắn', 'ổn cốc', 'ổn củ', 'ổn dao',
       'ổn decor', 'ổn dù', 'ổn dễ', 'ổn dịch', 'ổn em', 'ổn gia',
       'ổn giao', 'ổn gimbab', 'ổn giá', 'ổn giống', 'ổn giữ', 'ổn gà',
       'ổn gía', 'ổn gọi', 'ổn hai', 'ổn hi', 'ổn highly', 'ổn hình',
       'ổn hôm', 'ổn hơi', 'ổn hơn', 'ổn hương', 'ổn hết', 'ổn hộp',
       'ổn hợp', 'ổn kem', 'ổn khi', 'ổn khoai', 'ổn khoảng', 'ổn khu',
       'ổn khá', 'ổn không', 'ổn khẩu', 'ổn kimbab', 'ổn ko', 'ổn kín',
       'ổn lun', 'ổn luôn', 'ổn luônnnn', 'ổn là', 'ổn làm', 'ổn lá',
       'ổn lên', 'ổn lúc', 'ổn lại', 'ổn lấy', 'ổn lần', 'ổn lẩu',
       'ổn lắm', 'ổn lớp', 'ổn lựa', 'ổn mai', 'ổn matcha', 'ổn may',
       'ổn menu', 'ổn miếng', 'ổn mn', 'ổn mua', 'ổn mà', 'ổn màu',
       'ổn mì', 'ổn mình', 'ổn món', 'ổn mùa', 'ổn mấy', 'ổn mặc',
       'ổn mềm', 'ổn mọc', 'ổn mọi', 'ổn mỗi', 'ổn một', 'ổn mới',
       'ổn mở', 'ổn mức', 'ổ

In [16]:
detect_when_error_ana = ['13806594', '3756775', '2518027', '2470603', '2407997', '2656592', '3398288', '2578752', 
'2567874']

In [17]:
len(detect_when_error_ana)

9

In [18]:
df['RevId'] = df['RevId'].astype('str')

In [19]:
# df = df[~ df['RevId'].isin(detect_when_error_ana)]
list_rev_id = df['RevId'].to_list()
list_rating = df['Rating'].to_list()
for i in range(len(list_rev_id)):
    if list_rev_id[i] in detect_when_error_ana:
        if list_rating[i] == 1:
            list_rating[i] = 0
        else:
            list_rating[i] = 1
            
df['Rating'] = list_rating

In [20]:
df['Rating'].value_counts()

1.0    7151
0.0    1919
Name: Rating, dtype: int64

# Preprocessing

In [21]:
df['Comment'] = df['Comment'].str.lower()

In [22]:
emo = {"👉":" ","🤩":" rất mê ","👍🏻":" thích ", "👍":" thích ",
       '😄': " cười ", '😀':' cười ','😁':" thích ","😍":" yêu quá ",
       "😓":" chán ","😒":" khó ưa ","❤️":" yêu ","💔":" tổn thương ",
       "😆":" thích thú ","😘":" yêu thích ", "💋":" hôn ","🥳":" mừng rỡ ", 
       "☺️": " ", "😭": " khóc ròng ","😞":" buồn thiu ","😅":" cười toát mồ hôi ",
       "😌":" bí ẩn ","😂":" cười lăn lộn ","🤣":" cười lăn lộn ","🙄":" khó hiểu ", 
       "😑": " chán ", "😋":" thích thú ", "👌":" ổn ", "👌🏻":" ổn ","💯":" tuyệt vời ",
       "✅":" chất lượng ", "🤤":" ngon quá ", "😳": " ","😊":" thích thích ",
       "😚":" mê ", "😢":" buồn ", "😉": " ưng ", "🎉":" tuyệt ", "👏":" đáng khen ", 
       "🤬":" bức bội ", "🙃": " kì cục ", "💕":" yêu liền ", "👏🏻":" đáng khen ", 
       "🥺": " cảm động ","🙁":" buồn ","😕":" buồn ","😹":"cười to ","😻":" yêu quá ",
       "😎":" ngầu ","🙌🏻":" đồng tình","✌🏻":"ổn","🤨":" khó hiểu", "🤔":" suy nghĩ ",
       "😙":" thích thú ","😱":" bất ngờ ","😪":" buồn ngủ ","😝":" thích thú ",":(":" khóc ",
       "⭐⭐⭐⭐⭐":" tuyệt hảo ", '🔸': " ", '🤓': ' ', '📍': ' '}

In [23]:
import re

def preprocess(textdata):
    processedText = []
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    
    for text in textdata:
        check = re.search(r'([a-z])\1+',text)
        if check:
          if len(check.group())>2:
            text = re.sub(r'([a-z])\1+', lambda m: m.group(1), text, flags=re.IGNORECASE) #remove các ký tự kéo dài như hayyy,ngonnnn...

        text = text.strip() #loại dấu cách đầu câu
        # Replace all URls with 'URL'
        text = re.sub(urlPattern,' URL',text)
        # Replace comma,...
        text = re.sub(r"[\.,\?]+$-", "", text)
        # Replace all emojis.
        for emoji in emo.keys():
            if emo[emoji] != ' ':
                text = text.replace(emoji, " emo " + emo[emoji])
            else:
                text = text.replace(emoji," ")

        #Replace other dấu
        text = text.replace(",", " ").replace(".", " ") \
            .replace(";", " ").replace("“", " ") \
            .replace(":", " ").replace("”", " ") \
            .replace('"', " ").replace("'", " ") \
            .replace("!", " ").replace("?", " ") \
            .replace("-", " ").replace("?", " ").replace("\n",' ') \
            .replace("—",' ').replace('   ', ' ').replace('  ', ' ') \
            .replace(')','').replace('(','')
        
        dict_map = {
        "òa": "oà",
        "Òa": "Oà",
        "ÒA": "OÀ",
        "óa": "oá",
        "Óa": "Oá",
        "ÓA": "OÁ",
        "ỏa": "oả",
        "Ỏa": "Oả",
        "ỎA": "OẢ",
        "õa": "oã",
        "Õa": "Oã",
        "ÕA": "OÃ",
        "ọa": "oạ",
        "Ọa": "Oạ",
        "ỌA": "OẠ",
        "òe": "oè",
        "Òe": "Oè",
        "ÒE": "OÈ",
        "óe": "oé",
        "Óe": "Oé",
        "ÓE": "OÉ",
        "ỏe": "oẻ",
        "Ỏe": "Oẻ",
        "ỎE": "OẺ",
        "õe": "oẽ",
        "Õe": "Oẽ",
        "ÕE": "OẼ",
        "ọe": "oẹ",
        "Ọe": "Oẹ",
        "ỌE": "OẸ",
        "ùy": "uỳ",
        "Ùy": "Uỳ",
        "ÙY": "UỲ",
        "úy": "uý",
        "Úy": "Uý",
        "ÚY": "UÝ",
        "ủy": "uỷ",
        "Ủy": "Uỷ",
        "ỦY": "UỶ",
        "ũy": "uỹ",
        "Ũy": "Uỹ",
        "ŨY": "UỸ",
        "ụy": "uỵ",
        "Ụy": "Uỵ",
        "ỤY": "UỴ",
        }

        for i, j in dict_map.items():
            text = text.replace(i, j)
#             print(i,' ',j)
            
        
        map_phrase = {
            'ko định quay lại': 'không quay lại',
            'ko quay lại': 'không quay lại',
            'k bao h quay lại' : 'không quay lại',
            'chả dám quay lại': 'không quay lại',
            'k quay lại': 'không quay lại',
            'k ngon': 'không ngon',
        }
        
        for i, j in map_phrase.items():
            text = text.replace(i, j)

        processedText.append(text)


    return processedText

In [24]:
reviews = df["Comment"].tolist()
review = preprocess(reviews)

df['Comment'] = review

In [25]:
df = df[df['Comment'] != ' ']

In [26]:
df.shape

(9069, 6)

In [27]:
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

list_text = df['Comment'].to_list()
list_seq = []
for i in range(len(list_text)):
    words = rdrsegmenter.tokenize(list_text[i])[0]
  
    res = ''
    for j in range(len(words)):
        res = res + words[j] + ' '
    

    res = res.replace('___',' ').replace('   ', ' ').replace('  ', ' ').replace('\t',' ')
    list_seq.append(res)
    
df["Comment"] = list_seq

In [28]:
df

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
0,0,3839333,10106093.0,xôi dẻo đồ_ăn đậm vị hộp xôi được lót lá trông...,['https://images.foody.vn/res/g97/966781/s800/...,1.0
1,1,2824877,786914.0,gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,['https://images.foody.vn/res/g69/688413/s800/...,0.0
2,2,9816702,22467889.0,thời_tiết lạnh như này cả nhà rủ nhau đến lega...,['https://images.foody.vn/res/g72/715078/s800/...,1.0
3,3,2684585,1889449.0,em có đọc review thấy mng bảo trà sữa nướng đề...,['https://images.foody.vn/res/g90/895545/s800/...,0.0
4,4,2737987,8839942.0,đồ_ăn rất ngon nhà_hàng cũng rất đẹp tất_cả đề...,['https://images.foody.vn/res/g4/30186/s800/fo...,1.0
...,...,...,...,...,...,...
9068,9066,11236510,23044142.0,thực_sự mà nói thấy mọi người đánh_giá nhiều q...,['https://images.foody.vn/res/g94/930433/s800/...,1.0
9069,9067,1831645,10117823.0,lẩu thái tômyum đc 19 lò_đúc hbt hn theo đánh_...,['https://images.foody.vn/res/g16/150633/s800/...,1.0
9070,9068,2155495,11636069.0,ngay từ lúc đầu_tiên bước vào nhà_hàng đã được...,['https://images.foody.vn/res/g74/736118/s800/...,1.0
9071,9069,7357032,12208284.0,đặt ăn thử mà thấy ngón cá emo rất mê sẽ còn ủ...,['https://images.foody.vn/res/g104/1036659/s80...,1.0


In [29]:
df[df['Rating'].isnull()]

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
1392,1392,1446246,8728639.0,quán đẹp đẹp nhất trong những dingtea mình từn...,NaN,NaN


In [30]:
## => positive

df['Rating'] = df['Rating'].fillna(1)

In [31]:
df['Rating'].value_counts(dropna = False)

1.0    7150
0.0    1919
Name: Rating, dtype: int64

In [32]:
df.shape

(9069, 6)

In [33]:
list_len = []
list_text = df['Comment'].to_list()

for i in list_text:
    list_len.append(len(i.split()))

df['len'] = list_len

In [34]:
df['len'].describe([0.1 * i for i in range(1, 10)])

count    9069.000000
mean       77.950160
std        69.316249
min         1.000000
10%        15.000000
20%        24.000000
30%        34.000000
40%        46.000000
50%        60.000000
60%        78.000000
70%        96.000000
80%       116.000000
90%       162.000000
max       718.000000
Name: len, dtype: float64

In [35]:
len(df[df['len'] >= 256]['Comment'].to_list())

240

In [37]:
df[['len',"Comment","Rating"]].to_csv("duy_hihi_train.csv", encoding = 'utf-8-sig')

In [38]:
# df_test = pd.read_csv('Dataset/test.csv')
df_test = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/test.csv')

In [39]:
df_test['Comment'] = df_test['Comment'].fillna("None")

In [40]:
df_test['Comment'] = df_test['Comment'].str.lower()

In [41]:
cmts = df_test["Comment"].tolist()
cmts = preprocess(cmts)

df_test['Comment'] = cmts

In [42]:
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

list_text = df_test['Comment'].to_list()
list_seq = []
for i in range(len(list_text)):
    words = rdrsegmenter.tokenize(list_text[i])[0]
  
    res = ''
    for j in range(len(words)):
        res = res + words[j] + ' '
    

    res = res.replace('___',' ').replace('   ', ' ').replace('  ', ' ').replace('\t',' ')
    list_seq.append(res)
    
df_test["Comment"] = list_seq

In [43]:
list_len = []
list_text = df_test['Comment'].to_list()

for i in list_text:
    list_len.append(len(i.split()))

df_test['len'] = list_len

In [44]:
df_test

,Unnamed: 0,RevId,UserId,Comment,image_urls,len
0,0,781115,1326532,trà táo 35k cookie socola 38k nước ở đây bình_...,['https://images.foody.vn/res/g8/73091/s800/fo...,61
1,1,1219481,422306,hôm rồi trung_tâm mình tổ_chức noel party ở đâ...,['https://images.foody.vn/res/g1/33/s800/foody...,166
2,2,1703765,9779143,thịt gà của quán là nhất đấy đi ăn gọi liền 4 ...,['https://images.foody.vn/res/g66/659655/s800/...,137
3,3,4870346,12924388,hai đứa ăn xong đau bụng cả ngày chân gà ok nh...,['https://images.foody.vn/res/g78/772244/s800/...,15
4,4,2638711,1134279,mình vừa thử trưa nay điểm cộng đầu_tiên là bá...,['https://images.foody.vn/res/g74/737874/s800/...,58
...,...,...,...,...,...,...
5098,5098,1025826,132889,bún riêu ở đây ngon mà lúc_nào cũng đông thế_m...,['https://images.foody.vn/res/g10/92649/s800/f...,62
5099,5099,1278470,1691594,quán ngồi thoải_mái và cưc thích ❤ menu đa_dạn...,['https://images.foody.vn/res/g25/247296/s800/...,45
5100,5100,2565212,9989615,quá thất_vọng chất_lượng sản_phẩm ngày_càng đi...,['https://images.foody.vn/res/g89/888154/s800/...,53
5101,5101,3766155,17686862,giao hàng nhanh miếng băm chả đều ngon ko bị c...,['https://images.foody.vn/res/g13/128194/s800/...,15


In [45]:
df_test['len'].describe([0.1*i for i in range(1,10)])

count    5103.000000
mean       78.856359
std        70.427195
min         1.000000
10%        15.000000
20%        25.000000
30%        35.000000
40%        46.000000
50%        60.000000
60%        79.000000
70%        97.000000
80%       118.000000
90%       161.000000
max       863.000000
Name: len, dtype: float64

In [47]:
df_test[df_test['len']>=256].shape

(138, 6)

In [48]:
df_test

,Unnamed: 0,RevId,UserId,Comment,image_urls,len
0,0,781115,1326532,trà táo 35k cookie socola 38k nước ở đây bình_...,['https://images.foody.vn/res/g8/73091/s800/fo...,61
1,1,1219481,422306,hôm rồi trung_tâm mình tổ_chức noel party ở đâ...,['https://images.foody.vn/res/g1/33/s800/foody...,166
2,2,1703765,9779143,thịt gà của quán là nhất đấy đi ăn gọi liền 4 ...,['https://images.foody.vn/res/g66/659655/s800/...,137
3,3,4870346,12924388,hai đứa ăn xong đau bụng cả ngày chân gà ok nh...,['https://images.foody.vn/res/g78/772244/s800/...,15
4,4,2638711,1134279,mình vừa thử trưa nay điểm cộng đầu_tiên là bá...,['https://images.foody.vn/res/g74/737874/s800/...,58
...,...,...,...,...,...,...
5098,5098,1025826,132889,bún riêu ở đây ngon mà lúc_nào cũng đông thế_m...,['https://images.foody.vn/res/g10/92649/s800/f...,62
5099,5099,1278470,1691594,quán ngồi thoải_mái và cưc thích ❤ menu đa_dạn...,['https://images.foody.vn/res/g25/247296/s800/...,45
5100,5100,2565212,9989615,quá thất_vọng chất_lượng sản_phẩm ngày_càng đi...,['https://images.foody.vn/res/g89/888154/s800/...,53
5101,5101,3766155,17686862,giao hàng nhanh miếng băm chả đều ngon ko bị c...,['https://images.foody.vn/res/g13/128194/s800/...,15


In [49]:
df_test[['RevId','Comment','len']].to_csv('duy_hihi_test.csv', encoding = 'utf-8-sig')